# 🔥 YOLO11 Fire, Smoke & Fighting Detection Training Notebook

## Complete End-to-End Training Pipeline for InvEye

This notebook provides a comprehensive training pipeline for detecting:
- **Fire** 🔥
- **Smoke** 💨
- **Fighting/Violence** 👊

**Target Platform:** NVIDIA Jetson Orin Nano with DeepStream

---

### 📋 Table of Contents
1. [Environment Setup](#1-environment-setup)
2. [Dataset Download from Roboflow](#2-dataset-download)
3. [Dataset Exploration](#3-dataset-exploration)
4. [Model Training](#4-model-training)
5. [Model Validation](#5-model-validation)
6. [Model Testing & Inference](#6-inference)
7. [Export for Jetson Deployment](#7-export)
8. [Download Trained Model](#8-download)

---

> ⚠️ **IMPORTANT**: Run cells in order! Each section depends on the previous one.

---
## 1. Environment Setup <a name="1-environment-setup"></a>

### 1.1 Check GPU Availability

In [ ]:
# Check if GPU is available
!nvidia-smi

import torch
print(f"\n✅ PyTorch Version: {torch.__version__}")
print(f"✅ CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"✅ CUDA Version: {torch.version.cuda}")

### 1.2 Install Required Packages

In [ ]:
# Install ultralytics (YOLO11) and roboflow
!pip install -q ultralytics>=8.3.0
!pip install -q roboflow
!pip install -q opencv-python-headless
!pip install -q supervision

print("\n✅ All packages installed successfully!")

In [ ]:
# Verify ultralytics installation
from ultralytics import YOLO
import ultralytics

print(f"✅ Ultralytics Version: {ultralytics.__version__}")
ultralytics.checks()

### 1.3 Import Libraries

In [ ]:
import os
import glob
import shutil
import yaml
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from IPython.display import display, Image as IPImage, clear_output

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Initialize global variables
USE_CUSTOM_IMAGES = False
DATASET_PATH = None

print("✅ All libraries imported successfully!")

---
## 2. Dataset Download from Roboflow <a name="2-dataset-download"></a>

We'll download pre-annotated datasets for fire, smoke, and fighting detection from Roboflow.

> ⚠️ **Important:** Get your free API key from [Roboflow](https://roboflow.com/)

In [ ]:
from roboflow import Roboflow

# ========================================
# 🔑 ENTER YOUR ROBOFLOW API KEY HERE
# ========================================
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"  # Replace with your API key

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
print("✅ Connected to Roboflow!")

### 2.1 Download Fire & Smoke Dataset

We'll use a high-quality fire and smoke dataset from Roboflow Universe.

In [ ]:
# Option 1: Fire and Smoke Detection Dataset
# This is a popular dataset with good annotations

try:
    # Download fire detection dataset
    project = rf.workspace("roboflow-universe-projects").project("fire-detection-o7knn")
    fire_dataset = project.version(16).download("yolov8")
    DATASET_PATH = fire_dataset.location
    print(f"\n✅ Fire dataset downloaded to: {DATASET_PATH}")
except Exception as e:
    print(f"❌ Error downloading dataset: {e}")
    print("\n💡 Try the alternative dataset in the next cell.")

In [ ]:
# Alternative: Another fire/smoke dataset (run if above doesn't work)
# Only run this cell if the above cell failed

if DATASET_PATH is None:
    try:
        project = rf.workspace("-jwzpw").project("continuous_fire")
        fire_dataset = project.version(6).download("yolov8")
        DATASET_PATH = fire_dataset.location
        print(f"\n✅ Fire dataset downloaded to: {DATASET_PATH}")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("✅ Dataset already downloaded. Skipping alternative.")

### 2.2 Download Fighting/Violence Dataset (Optional)

In [ ]:
# Download fighting/violence detection dataset
# This is optional - run only if you want to detect fighting as well

INCLUDE_FIGHTING = False  # Set to True to include fighting detection

if INCLUDE_FIGHTING:
    try:
        project = rf.workspace("violence-detection").project("fight-detection-dataset")
        fight_dataset = project.version(1).download("yolov8")
        print(f"\n✅ Fighting dataset downloaded to: {fight_dataset.location}")
    except Exception as e:
        print(f"⚠️ Could not download fighting dataset: {e}")
        print("Continuing with fire detection only...")
        INCLUDE_FIGHTING = False
else:
    print("ℹ️ Skipping fighting dataset (set INCLUDE_FIGHTING = True to include)")

### 2.3 Alternative: Use Your Own Images

If you have your own fire incident images, use this section to set up the dataset structure.

In [ ]:
# ============================================
# ALTERNATIVE: Using Your Own Custom Images
# ============================================

# Set to True ONLY if you want to use your own images
USE_CUSTOM_IMAGES = False

if USE_CUSTOM_IMAGES:
    # Create dataset directory structure
    custom_dataset_path = "/content/custom_fire_dataset"
    
    for split in ['train', 'valid', 'test']:
        os.makedirs(f"{custom_dataset_path}/images/{split}", exist_ok=True)
        os.makedirs(f"{custom_dataset_path}/labels/{split}", exist_ok=True)
    
    print("\n📁 Created dataset structure:")
    print(f"   {custom_dataset_path}/")
    print("   ├── images/")
    print("   │   ├── train/   <- Put 70% of your images here")
    print("   │   ├── valid/   <- Put 20% of your images here")
    print("   │   └── test/    <- Put 10% of your images here")
    print("   └── labels/")
    print("       ├── train/   <- Corresponding .txt label files")
    print("       ├── valid/")
    print("       └── test/")
    
    # Create data.yaml
    data_yaml = {
        'path': custom_dataset_path,
        'train': 'images/train',
        'val': 'images/valid',
        'test': 'images/test',
        'names': {
            0: 'fire',
            1: 'smoke'
        }
    }
    
    with open(f"{custom_dataset_path}/data.yaml", 'w') as f:
        yaml.dump(data_yaml, f)
    
    DATASET_PATH = custom_dataset_path
    print(f"\n✅ Created data.yaml at {custom_dataset_path}/data.yaml")
    print("\n⚠️ Remember: Each image needs a corresponding .txt label file!")
    print("   Format: class_id x_center y_center width height (normalized 0-1)")
else:
    print("ℹ️ Using Roboflow dataset (not custom images)")

---
## 3. Dataset Exploration <a name="3-dataset-exploration"></a>

> ⚠️ **Make sure Section 2.1 ran successfully before continuing!**

In [ ]:
# Verify dataset path is set
if DATASET_PATH is None:
    print("❌ ERROR: DATASET_PATH is not set!")
    print("")
    print("Please go back and run Section 2.1 (Download Fire & Smoke Dataset) first.")
    print("Make sure the download completed successfully.")
    raise ValueError("Dataset not downloaded. Run Section 2.1 first.")
else:
    print(f"✅ Dataset Path: {DATASET_PATH}")
    print("\n📊 Dataset Contents:")
    !ls -la {DATASET_PATH}

In [ ]:
# Load and display data.yaml
data_yaml_path = f"{DATASET_PATH}/data.yaml"

with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("📋 Dataset Configuration (data.yaml):")
print("=" * 50)
for key, value in data_config.items():
    print(f"  {key}: {value}")

# Get class names
class_names = data_config.get('names', {})
num_classes = len(class_names)
print(f"\n📊 Number of Classes: {num_classes}")
print(f"📊 Class Names: {class_names}")

In [ ]:
# Count images in each split
def count_images(path):
    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    count = 0
    for ext in extensions:
        count += len(glob.glob(os.path.join(path, ext)))
    return count

# Handle different folder structures
train_images = f"{DATASET_PATH}/train/images" if os.path.exists(f"{DATASET_PATH}/train/images") else f"{DATASET_PATH}/images/train"
valid_images = f"{DATASET_PATH}/valid/images" if os.path.exists(f"{DATASET_PATH}/valid/images") else f"{DATASET_PATH}/images/valid"
test_images = f"{DATASET_PATH}/test/images" if os.path.exists(f"{DATASET_PATH}/test/images") else f"{DATASET_PATH}/images/test"

print("📊 Dataset Statistics:")
print("=" * 50)
if os.path.exists(train_images):
    print(f"  Training Images:   {count_images(train_images)}")
if os.path.exists(valid_images):
    print(f"  Validation Images: {count_images(valid_images)}")
if os.path.exists(test_images):
    print(f"  Test Images:       {count_images(test_images)}")

In [ ]:
# Visualize sample images from dataset
def visualize_samples(image_folder, num_samples=6):
    """Display sample images from the dataset"""
    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    all_images = []
    for ext in extensions:
        all_images.extend(glob.glob(os.path.join(image_folder, ext)))
    
    if not all_images:
        print(f"No images found in {image_folder}")
        return
    
    samples = random.sample(all_images, min(num_samples, len(all_images)))
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(samples):
        if idx >= 6:
            break
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(os.path.basename(img_path)[:30], fontsize=8)
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle("🔥 Sample Training Images", fontsize=14, y=1.02)
    plt.show()

print("🖼️ Sample Training Images:")
visualize_samples(train_images)

---
## 4. Model Training <a name="4-model-training"></a>

### 4.1 Training Configuration

In [ ]:
# ============================================
# 🔧 TRAINING CONFIGURATION
# ============================================

# Model Selection (choose one)
# Options: 'yolo11n.pt', 'yolo11s.pt', 'yolo11m.pt', 'yolo11l.pt', 'yolo11x.pt'
# Smaller = Faster but less accurate | Larger = Slower but more accurate
MODEL_SIZE = 'yolo11n.pt'  # nano - best for Jetson

# Training Parameters
EPOCHS = 100          # Number of training epochs (increase for better results)
BATCH_SIZE = 16       # Batch size (reduce if OOM error)
IMAGE_SIZE = 640      # Input image size
PATIENCE = 50         # Early stopping patience
WORKERS = 4           # Number of data loading workers

# Optimization
OPTIMIZER = 'auto'    # Options: 'SGD', 'Adam', 'AdamW', 'auto'
LR0 = 0.01           # Initial learning rate
LRF = 0.01           # Final learning rate factor

# Augmentation (data augmentation settings)
AUGMENT = True
MOSAIC = 1.0         # Mosaic augmentation probability
MIXUP = 0.0          # Mixup augmentation probability
COPY_PASTE = 0.0     # Copy-paste augmentation probability

# Project Settings
PROJECT_NAME = 'fire_detection'
EXPERIMENT_NAME = 'yolo11_fire_smoke'

print("✅ Training configuration set!")
print(f"\n📋 Configuration Summary:")
print(f"   Model: {MODEL_SIZE}")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Image Size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"   Dataset: {DATASET_PATH}")

### 4.2 Start Training

In [ ]:
# Load pre-trained YOLO11 model
model = YOLO(MODEL_SIZE)

print(f"✅ Loaded pre-trained {MODEL_SIZE} model")
print(f"\n🚀 Starting training...")
print("=" * 60)

In [ ]:
# Train the model
results = model.train(
    data=data_yaml_path,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMAGE_SIZE,
    patience=PATIENCE,
    workers=WORKERS,
    optimizer=OPTIMIZER,
    lr0=LR0,
    lrf=LRF,
    augment=AUGMENT,
    mosaic=MOSAIC,
    mixup=MIXUP,
    copy_paste=COPY_PASTE,
    project=PROJECT_NAME,
    name=EXPERIMENT_NAME,
    exist_ok=True,
    pretrained=True,
    verbose=True,
    seed=42,
    device=0,  # Use GPU 0
    cache=True,  # Cache images for faster training
    amp=True,  # Use automatic mixed precision
    plots=True,  # Generate training plots
    save=True,
    save_period=10  # Save checkpoint every 10 epochs
)

print("\n" + "=" * 60)
print("✅ Training completed!")

### 4.3 View Training Results

In [ ]:
# Display training results
RESULTS_DIR = f"{PROJECT_NAME}/{EXPERIMENT_NAME}"

print(f"📁 Results saved to: {RESULTS_DIR}")
print("\n📊 Training artifacts:")
!ls -la {RESULTS_DIR}

In [ ]:
# Display training curves
results_png = f"{RESULTS_DIR}/results.png"
if os.path.exists(results_png):
    print("📈 Training Curves:")
    display(IPImage(filename=results_png))
else:
    print("⚠️ Results image not found. Training may still be in progress.")

In [ ]:
# Display confusion matrix
confusion_matrix_png = f"{RESULTS_DIR}/confusion_matrix.png"
if os.path.exists(confusion_matrix_png):
    print("📊 Confusion Matrix:")
    display(IPImage(filename=confusion_matrix_png))

In [ ]:
# Display sample predictions from validation
val_batch_png = f"{RESULTS_DIR}/val_batch0_pred.png"
if os.path.exists(val_batch_png):
    print("🔍 Sample Validation Predictions:")
    display(IPImage(filename=val_batch_png))

---
## 5. Model Validation <a name="5-model-validation"></a>

In [ ]:
# Load best trained model
BEST_MODEL_PATH = f"{RESULTS_DIR}/weights/best.pt"

if os.path.exists(BEST_MODEL_PATH):
    best_model = YOLO(BEST_MODEL_PATH)
    print(f"✅ Loaded best model from: {BEST_MODEL_PATH}")
else:
    print("⚠️ Best model not found. Using last trained model.")
    best_model = model

In [ ]:
# Validate the model
print("🔍 Running validation...")
val_results = best_model.val(
    data=data_yaml_path,
    batch=BATCH_SIZE,
    imgsz=IMAGE_SIZE,
    split='val',
    plots=True,
    save_json=True
)

print("\n📊 Validation Metrics:")
print("=" * 50)
print(f"  mAP50:      {val_results.box.map50:.4f}")
print(f"  mAP50-95:   {val_results.box.map:.4f}")
print(f"  Precision:  {val_results.box.mp:.4f}")
print(f"  Recall:     {val_results.box.mr:.4f}")

In [ ]:
# Per-class metrics
print("\n📊 Per-Class Performance:")
print("=" * 50)
for i, class_name in enumerate(class_names.values()):
    if i < len(val_results.box.ap50):
        print(f"  {class_name}:")
        print(f"    - AP50: {val_results.box.ap50[i]:.4f}")
        print(f"    - AP:   {val_results.box.ap[i]:.4f}")

---
## 6. Model Testing & Inference <a name="6-inference"></a>

In [ ]:
# Test on sample images
print("🔍 Running inference on test images...")

# Get test images
test_folder = test_images if os.path.exists(test_images) else valid_images
test_image_files = glob.glob(os.path.join(test_folder, '*.jpg')) + \
                   glob.glob(os.path.join(test_folder, '*.png'))

if test_image_files:
    # Run inference on a few test images
    sample_images = random.sample(test_image_files, min(4, len(test_image_files)))
    
    predictions = best_model.predict(
        source=sample_images,
        conf=0.25,
        iou=0.45,
        save=True,
        project=PROJECT_NAME,
        name='test_predictions'
    )
    
    print(f"\n✅ Predictions saved to: {PROJECT_NAME}/test_predictions")
else:
    print("⚠️ No test images found.")

In [ ]:
# Display prediction results
pred_folder = f"{PROJECT_NAME}/test_predictions"
if os.path.exists(pred_folder):
    pred_images = glob.glob(os.path.join(pred_folder, '*.jpg')) + \
                  glob.glob(os.path.join(pred_folder, '*.png'))
    
    if pred_images:
        fig, axes = plt.subplots(1, min(4, len(pred_images)), figsize=(20, 5))
        if len(pred_images) == 1:
            axes = [axes]
        
        for idx, img_path in enumerate(pred_images[:4]):
            img = Image.open(img_path)
            axes[idx].imshow(img)
            axes[idx].set_title(f"Prediction {idx+1}")
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.suptitle("🔥 Fire Detection Predictions", fontsize=14, y=1.02)
        plt.show()

---
## 7. Export for Jetson Deployment <a name="7-export"></a>

Export the trained model in various formats for deployment.

In [ ]:
# Export to ONNX (for DeepStream and general deployment)
print("📦 Exporting model to ONNX format...")

onnx_path = best_model.export(
    format='onnx',
    imgsz=IMAGE_SIZE,
    half=False,
    simplify=True,
    opset=12
)

print(f"\n✅ ONNX model exported to: {onnx_path}")

In [ ]:
# Summary of all exported files
print("\n📦 Exported Model Files:")
print("=" * 60)

weights_dir = f"{RESULTS_DIR}/weights"
if os.path.exists(weights_dir):
    for file in os.listdir(weights_dir):
        filepath = os.path.join(weights_dir, file)
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"  📄 {file} ({size_mb:.2f} MB)")

---
## 8. Download Trained Model <a name="8-download"></a>

In [ ]:
# Create a zip file with all trained models
import zipfile

zip_filename = 'fire_detection_models.zip'

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add best.pt
    if os.path.exists(f"{weights_dir}/best.pt"):
        zipf.write(f"{weights_dir}/best.pt", 'best.pt')
    
    # Add last.pt
    if os.path.exists(f"{weights_dir}/last.pt"):
        zipf.write(f"{weights_dir}/last.pt", 'last.pt')
    
    # Add ONNX if exists
    onnx_file = f"{weights_dir}/best.onnx"
    if os.path.exists(onnx_file):
        zipf.write(onnx_file, 'best.onnx')
    
    # Add results
    if os.path.exists(f"{RESULTS_DIR}/results.png"):
        zipf.write(f"{RESULTS_DIR}/results.png", 'training_results.png')
    
    # Add data.yaml for reference
    zipf.write(data_yaml_path, 'data.yaml')

print(f"\n✅ Created: {zip_filename}")
print(f"   Size: {os.path.getsize(zip_filename) / (1024 * 1024):.2f} MB")

In [ ]:
# Download the zip file (for Google Colab)
from google.colab import files

print("📥 Downloading trained models...")
files.download(zip_filename)
print("\n✅ Download started!")

In [ ]:
# Alternative: Download individual files
print("📥 Download individual files:")
print("\n1. Best Model (PyTorch):")
files.download(f"{weights_dir}/best.pt")

---
## 🎉 Training Complete!

### Summary

You now have a trained YOLO11 fire detection model. Here's what was created:

| File | Description | Use Case |
|------|-------------|----------|
| `best.pt` | Best performing model | General PyTorch inference |
| `last.pt` | Last epoch model | Backup/resume training |
| `best.onnx` | ONNX format | Cross-platform deployment |

### Next Steps for Jetson Deployment

1. **Copy `best.pt` to your Jetson device**
2. **Convert to TensorRT on Jetson** (for best performance):
   ```bash
   yolo export model=best.pt format=engine device=0 half=True
   ```
3. **Integrate with DeepStream** using the generated `.engine` file

### Model Performance Tips

- If accuracy is low, try:
  - Training for more epochs (200-300)
  - Using a larger model (`yolo11s.pt` or `yolo11m.pt`)
  - Adding more training data
  - Fine-tuning augmentation parameters

- If inference is slow on Jetson:
  - Use `yolo11n.pt` (nano) model
  - Enable FP16 (`half=True`)
  - Reduce input size to 416 or 320